In [1]:
import pandas as pd
import numpy as np
import math

# Assignment 1 (Random Forest)

* In this project you are given a dataset of housing housing price prediction. 
Dataset description is found in the given datasets.

* The goal of the project is to predict the price of a house given its attributes. 
Therefore, the problem is a regression task. 

* You need to build a random forest that consists of multiple decision trees (for regression) from the given training data set. Then, apply it on the test set and submit your code to generate predictions.
You need to build the random forest and decision trees from scratch. (I.e., it is not allowed to use existing machine learning libraries or packages such as sklearn.)

* You may use any programming language/environment of your choice, but you are required to submit the complete source code to produce the output
If you use anything other than jupyter notebook, submit an executable and run that from the main function of the jupyter notebook so that the prediction generation is automated. We can provide assistance with this.
The output (a single file with the predictions for each test instance) must be generated automatically using the approach implemented by you. Submitting predictions/code from any other source (Internet, another student, etc.) is considered cheating and will result in immediate disqualification (i.e., dismissal from the course).

## Part 1: Preprocessing and dataset analysis (20 points)

* The given dataset is quite complex, it has many attributes, and not all of them are useful! 
Training on such dataset results in a bad accuracy. And this is exactly the point! 

* "Understanding the question is half the answer". In data mining, understanding the dataset is half the answer! 

* In part 1 you need to analyze the dataset and make it clean. 
 

### Load the dataset and explore (5 points)

* Load the dataset, view the dataset and the shape of it, 

In [3]:
#Load the data
train_data_file = 'housing_price_train.csv'
test_data_file = 'housing_price_test.csv'

train_data = pd.read_csv(train_data_file)
test_data = pd.read_csv(test_data_file)
cols = train_data.columns

print(train_data.shape, test_data.shape)
print(cols)
train_data.head()

(1460, 81) (1560, 80)
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'Ga

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Clean the dataset (10 points)

* We cannot train on a 'dirty' dataset! There are duplicated, Null, and missing values that you need to take care of!

* Drop all columns which have null values >= 70 % and drop all rows which have null values >= 70 %.

* You need to take care of categorial data!

In [4]:
def PreprocessingData(db, threshold = 70):
    """
        TODO
        Show NULL values for each columns in dataset and thier percentage
        Drop all columns which have null values >= 70 % 
        Drop all rows which have null values >= 70 %
        Fill all missing values with the (mean, mode)

    """
    # 1. Show null values and their percentage
    columns_with_null_values = (db.isna().sum().loc[lambda x: x > 0]).to_frame()
    columns_with_null_values["Percentage"] = columns_with_null_values.apply(lambda x: (x / len(db)) * 100)
    print("\nColumns with null values: ")
    print(columns_with_null_values)
    
    
    # 2. Drop columns with more than 70% null values
    columns_to_remove = (columns_with_null_values["Percentage"].loc[lambda x: x > threshold]).index
    print("\nColumns to Remove: ")
    print(columns_to_remove)
    db.drop(columns_to_remove, axis = 'columns', inplace = True)
    
    # 3. Drop all rows with more than 70% null values
    min_non_null_values = len(db.columns) - int((threshold * len(db.columns)) / 100)
    print("\nThreshold for at least 70% null values in a row: ")
    print(min_non_null_values)
    # No rows have more than 70% null values
    if len(db.isna().sum(axis = 1).loc[lambda x: x > min_non_null_values]) == 0:
        print("\nNo rows have more than 70% Null Values.")
        print(db.isna().sum(axis = 1).loc[lambda x: x > min_non_null_values])
    else:
        db.dropna(axis = 'index', thresh = min_non_null_values, inplace = True)
        print("\nShape after dropping rows with at least 70% null values: ")
        print(db.shape)
    
    # 4. Fill missing values (mean for numerical values and mode for categorical)
    numeric_columns = list(db.select_dtypes(['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns)
    categorical_columns = list(db.select_dtypes(include = ['object', 'category']).columns)
    
    for col in db.columns:
        if col in ['Id']:
            continue
        elif col in numeric_columns:
            db[col].fillna(db[col].mean(), inplace=True)
        elif col in categorical_columns:
            db[col].fillna(db[col].mode()[0], inplace=True)
            
    print("\nNull Values after cleaning up the dataset: ")
    print(db.isna().sum(axis=1).loc[lambda x: x > 0])
    print(db.isna().sum(axis=0).loc[lambda x: x > 0])

    return db

In [5]:
train_data = PreprocessingData(train_data)
test_data = PreprocessingData(test_data)


Columns with null values: 
                 0  Percentage
LotFrontage    259   17.739726
Alley         1369   93.767123
MasVnrType       8    0.547945
MasVnrArea       8    0.547945
BsmtQual        37    2.534247
BsmtCond        37    2.534247
BsmtExposure    38    2.602740
BsmtFinType1    37    2.534247
BsmtFinType2    38    2.602740
Electrical       1    0.068493
FireplaceQu    690   47.260274
GarageType      81    5.547945
GarageYrBlt     81    5.547945
GarageFinish    81    5.547945
GarageQual      81    5.547945
GarageCond      81    5.547945
PoolQC        1453   99.520548
Fence         1179   80.753425
MiscFeature   1406   96.301370

Columns to Remove: 
Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

Threshold for at least 70% null values in a row: 
24

No rows have more than 70% Null Values.
Series([], dtype: int64)

Null Values after cleaning up the dataset: 
Series([], dtype: int64)
Series([], dtype: int64)

Columns with null values: 
                 0  P

### Correlations! (5 points)

* Now we have a clean dataset, but not all attributes are useful! 

* Display the corrlation between all features and the sales price. This will show you which feature affects sales price more. You may use *corr()* function. 

* Choose the most correlated features, and remove others. 

In [6]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


The RandomForest Algorithm can work well with the numerical columns but for categorical columns we need to convert them to numerical columns somehow. One common method to do so is to make each category into a column and items / rows with that category will have 1 in the respective column and 0 in others.

We can use get_dummies method from pandas to do so.

In [7]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [8]:
# For correlation we will use 20% as the threshold.
threshold = 0.20

In [9]:
# Selecting SalePrice column since we only want the correlation between Columns and SalePrice
# And taking absolute value cause there could be a negative correlation between Attribute and SalePrice
train_data_correlation = train_data.corr()['SalePrice'].abs()
columns_to_drop = train_data_correlation.loc[lambda x: x < threshold].index.to_list()
train_data_cleaned = train_data.drop(columns_to_drop, axis = 1)

train_data_cleaned.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageType_Attchd,GarageType_BuiltIn,GarageType_Detchd,GarageFinish_Fin,GarageFinish_Unf,PavedDrive_N,PavedDrive_Y,SaleType_New,SaleType_WD,SaleCondition_Partial
0,65.0,8450,7,2003,2003,196.0,706,150,856,856,...,1,0,0,0,0,0,1,0,1,0
1,80.0,9600,6,1976,1976,0.0,978,284,1262,1262,...,1,0,0,0,0,0,1,0,1,0
2,68.0,11250,7,2001,2002,162.0,486,434,920,920,...,1,0,0,0,0,0,1,0,1,0
3,60.0,9550,7,1915,1970,0.0,216,540,756,961,...,0,0,1,0,1,0,1,0,1,0
4,84.0,14260,8,2000,2000,350.0,655,490,1145,1145,...,1,0,0,0,0,0,1,0,1,0


In [10]:
# Next we remove the same columns from test_data that we just dropped from train_data with low enough correlation
training_cols = train_data_cleaned.columns.tolist()
test_cols = test_data.columns.tolist()

dropped_training_cols = np.setdiff1d(test_cols, training_cols)
test_data_cleaned = test_data.drop(dropped_training_cols, axis = 1, inplace = False)
print(train_data_cleaned.shape, test_data_cleaned.shape)

(1460, 68) (1560, 67)


In [11]:
# To make indexing easier for us later we rearrange the columns in train_data for SalePrice to be the last column
list_of_columns = list(train_data_cleaned.columns)
list_of_columns.remove('SalePrice')
list_of_columns.append('SalePrice')
train_data_cleaned = train_data_cleaned.reindex(columns = list_of_columns)
print(train_data_cleaned.iloc[:,:-1].columns.to_list())
print(train_data_cleaned.iloc[:,-1].name)

['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'MSZoning_RL', 'MSZoning_RM', 'LotShape_IR1', 'LotShape_Reg', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_StoneBr', 'HouseStyle_2Story', 'RoofStyle_Gable', 'RoofStyle_Hip', 'Exterior1st_VinylSd', 'Exterior2nd_VinylSd', 'MasVnrType_None', 'MasVnrType_Stone', 'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA', 'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_PConc', 'BsmtQual_Ex', 'BsmtQual_Gd', 'BsmtQual_TA', 'BsmtExposure_Gd', 'BsmtExposure_No', 'BsmtFinType1_GLQ', 'HeatingQC_Ex', 'HeatingQC_TA', 'CentralAir_N', 'CentralAir_Y', 'Electrical_SBrkr', 'KitchenQual_Ex', 'KitchenQual_Gd', 'KitchenQual_TA', 'FireplaceQu_Ex', 'GarageType_Attchd', 'GarageType_BuiltIn', 'Garag

## Part 2: Decision Tree (45 points)
#### Building a Decision Tree:
A Decision tree consists of nodes connected by edges. A decision tree is typically, a binary tree, which has the following properties:
- One node is marked as Root node
- Every node other than the root has a parent node
- Each node can have at most 2 child nodes (left edge & right edge)
- Leaf node is the node which contains pure data or when we reach to the maximum depth 

To create the decision tree model for scratch you need to create two classes (a class for the node, for example "class DecisionNode():" and a class for Decision Tree model, for example "class RegressionDecisionTree():")


1- DecisionNode class used to save some values for each node we do the spliting on it until we reach the leaf node
so we will save the following values for the node:
- feature: feature index.
- threshold: the value we used to split the data on.
- value: the average value for the leaf node.
- True_Branch: if the condition is true.
- False_Branch: if the condition is false.

In [12]:
class DecisionNode():
    def __init__(self, feature_idx=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature_idx = feature_idx # index of the feature that is used
        self.threshold = threshold     # threshold value for feature when making the decision
        self.value = value # Average value if the node is a leaf in the tree
        self.true_branch = true_branch # the node we go to if decision returns True
        self.false_branch = false_branch # the node we go to if decision returns False

# Decision Tree Class
This Class consists the following functions:
<ol>
<li> <b>build_tree</b>: used to create the decision tree nodes</li> 
<li> <b>calc_variance_reduction</b> : measure the impurity by using variance reduction measure (like MSE) </li> 
the function takes three parameters (parentRec: the records for the target before split,and the left and right records after splitting. This function used to measure the impurity for each node and decide if we will split or not.
<li> <b>majority_vote</b>: used to calculate values for the leaf nodes records which equal to the mean of these records.</li> 
<li><b>split_by_feature</b>: this function take the feature and the threshold and check if the feature is numerical so it split the records into two node (true which is the left edge and false which is the right edge)
if the feature is categorical so it split where the values equal to the threshold</li>
<li> <b>fit</b>: Used to train the dataset after spliting the data into two part x: features, y: target</li>
<li><b>predict_value</b>: used to predict the value for each record, it is a recursive method to find the leaf node that corresponds to prediction
<li><b>predict</b>: take all records for the test data and iterate into each record to predit the y(target) value and save the result into a prediction list. 

In [13]:
class RegressionDecisionTree():
    # constructor
    def __init__(self, min_VarianceReduction=1e-7, max_depth=5):        
        self.root = None # root of this tree
        self.min_VarianceReduction = min_VarianceReduction # minimum VarianceReduction to allow splitting
        # used to stopping conductions
        self.max_depth = max_depth # maximum depth the tree grows to
 

    # used to create the decision tree nodes
    def build_tree(self, X, y, current_depth=0):
        # we will use decision dictionary to save the feature and the threshold we build the tree on 
        decision = None
        # we will use subtrees dictionary to save the feature and the threshold we build the tree on 
        subtrees = None
        largest_variance_Reduction = 0
        # add y as last column of X
        df = pd.concat((X, y), axis=1)
        n_rows, n_features = X.shape
        if current_depth <= self.max_depth:
            # iterate through every feature
            for feature_idx in range(n_features):
                # values of that column
                feature_values = X.iloc[:, feature_idx]                
                unique_values = feature_values.unique()                
                for threshold in unique_values:
                    X_trueEdge, X_falseEdge = self.split_by_feature(df, feature_idx, threshold)
                    if len(X_trueEdge) > 0 and len(X_falseEdge) > 0:
                        y_true = X_trueEdge.iloc[:,-1]
                        y_false = X_falseEdge.iloc[:,-1]                        
                        # Calculate impurity
                        VarianceRed = self.Calc_variance_reduction(y, y_true, y_false)
                        # Keep track of which feature gave the largest information gain
                        if VarianceRed > largest_variance_Reduction:
                            largest_variance_Reduction = VarianceRed
                            decision = {"feature_idx":feature_idx, "threshold":threshold}
                            subtrees = {"X_true":X_trueEdge.iloc[:,:-1],
                                        "y_true":y_true,
                                        "X_false":X_falseEdge.iloc[:,:-1],
                                        "y_false":y_false}

        # we will construct new branch of tree if the variance_Reduction is larger than minimum variance_Reduction that we've defined
        if largest_variance_Reduction > self.min_VarianceReduction:
            true_branch = self.build_tree(subtrees["X_true"], subtrees["y_true"], current_depth+1)
            false_branch = self.build_tree(subtrees["X_false"], subtrees["y_false"], current_depth+1)
            return DecisionNode(feature_idx=decision["feature_idx"], threshold=decision["threshold"], true_branch=true_branch, false_branch=false_branch)

        # at leaf node we calculate the mean for the records
        leaf_value = self.majority_vote(y)
        return DecisionNode(value=leaf_value)
                        
    # measure the impurity by using variance reduction measure (like MSE)
    # left_edgeRec= True edge: where condition is true
    # Right_edgeRec= False edge: where condition is false
    def Calc_variance_reduction(self, parentRec, left_edgeRec, Right_edgeRec):   
        n = len(parentRec)
        left_weight = len(left_edgeRec) / n
        right_weight = len(Right_edgeRec) / n
        
        parent_variance = parentRec.var()
        left_variance = left_edgeRec.var()
        right_variance = Right_edgeRec.var()
        
        # return the VarReduction = variance for parent - (Weight * var(leftEdge) + Weight * var(RightEdge)    
        return (parent_variance - ((left_weight * left_variance) + (right_weight * right_variance)))
    
    def majority_vote(self, y):
        # Just taken from the Helping material
        return y.value_counts().idxmax()
    
    def split_by_feature(self, X, feature_idx, threshold):
        # split the data into left_edge & right_edge depends one specified feature and the threshold
        condition_satisfied_indexes = (X.iloc[:, feature_idx] >= threshold) == True
        condition_not_satisfied_indexes = (X.iloc[:,feature_idx] >= threshold) == False
        
        left_tree = X[condition_satisfied_indexes]
        right_tree = X[condition_not_satisfied_indexes]
        # return left & right edges
        return(left_tree, right_tree)

    
    # Used to train the dataset after spliting the data into x: features, y: target
    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict_value(self, xTest, tree = None):
        # Just taken from the Helping material
        # recursive method to find the leaf node that corresponds to prediction
        if tree is None:
            tree = self.root
        if tree.value is not None:
            return tree.value

        feature_value = xTest[tree.feature_idx]
        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.true_branch
        elif feature_value == tree.threshold:
            branch = tree.true_branch

        return self.predict_value(xTest, branch)

    # to predict the value we need to pass the all records for features and we save the prediction for each records into a list
    def predict(self, XTest):
        y_pred = []
        for idx, row in XTest.iterrows():           
            y_pred.append(self.predict_value(row.values))
        return y_pred


- To Check the Accuracy for our prediction we use CalcAccuracy function which take the actual values for the test dataset and the predicted values and apply the RMSE formula.

In [14]:
def CalcAccuracy(Actual_Y, Predicted_y):
    return math.sqrt(np.square(np.subtract(Actual_Y, Predicted_y)).mean())

- Build decision tree model
- Fit the model
- Predict the values from test data 

In [15]:
decission_tree = RegressionDecisionTree()
decission_tree.fit(train_data_cleaned.iloc[:,:-1], train_data_cleaned.iloc[:,-1])

In [16]:
# Selecting only until 1459 because test_data has some additional rows than the labels given in submissions.csv
predicted_y = (decission_tree.predict(test_data_cleaned))[:1459]

In [17]:
test_labels_file = 'sample_submission.csv'
test_labels = pd.read_csv(test_labels_file)

In [18]:
CalcAccuracy(list(test_labels["SalePrice"]), predicted_y)

46898.04901338459

### So our model is off by around 47000 NOK value

## Part 3: Random Forest (20 points)
#### Random forest class
- the Class consist of the following functions:
<ul>
    <li>Constructor: consists of the subset data (Training & Testing) dataset after preprocessing and a list of deciceion tree objects </li>    
    <li>Subsampling: Bagging we will take random sample with replacement for the Training dataset </li>
    <li>build_model: first make subsample for the training dataset, then split the data into featurespart(X) and targetpart(Y), then take 10 samples of the feature part, finally build the decision tree (fit), this function take the number of DT that we want to build</li>
    <li>predict: take the test dataset and make the prediction for the target field in all the tree in the random forest then take the mean for the prediction in each tree, finally add the mean of prediction to a list of predition </li>

</ul>

In [19]:
class RF(object):
    def __init__(self):
        self.Traindata = None  # training data set (loaded into memory)
        self.Testdata = None  # Test data set for prediction        
        self.trees = []  # list of decision trees 
          

     # This function generate a subsample with replacement
    def __subsampling(self, train_set, sample_size_ratio):       
        sample_number = round(len(train_set) * sample_size_ratio)
        return train_set.sample(sample_number, replace = True)
        
    def build_model(self, train_set, sample_size_ratio, number_of_trees):
        for i in range(number_of_trees):
            TrainingSample = self.__subsampling(train_set, sample_size_ratio)
            tree = RegressionDecisionTree()
            tree.fit(TrainingSample.iloc[:,:-1], TrainingSample.iloc[:,-1])
            self.trees.append(tree)
            
    def predict(self, test_set):
        n = len(test_set)
        m = len(self.trees)
        forest_pred = np.zeros((n, m))
        counter = 0
        result = []
        for tree in self.trees:
            forest_pred[:, counter] = tree.predict(test_set)
            counter += 1

        result = np.mean(forest_pred, axis = 1)
        return result

### Create Random Forest

* Create 10 Decision Tree in the randomforest
* Train the random forest with the dataset
* Use the created random forest to predict the test dataset 

In [21]:
forest = RF()
# This takes a long time to run
# I am using a very small sample_size_ratio so that the random forest trains faster
forest.build_model(train_data_cleaned, 0.1, 10)

In [22]:
forest_predicted_y = (forest.predict(test_data_cleaned))[:1459]

In [23]:
CalcAccuracy(list(test_labels["SalePrice"]), forest_predicted_y)

38912.07132675759

## Part4: Comparison! (15 points)

Now that you have finished implementing your Random Forest, it's time for some experiments and analysis! 

* Use the Random Forest in the scikit-learn library and train it on the same dataset. 

* Compare the accuracy given by your Random Forest to the scikit-learn one. 

* Increase the number of trees in your Random Forest. Does it improve the accuracy? 

* Make a table for comparing your Random Forest accuracy with different number of trees with the scikit-learn one. What is your conclusion? 

In [27]:
from sklearn.ensemble import RandomForestClassifier
sklearn_forest = RandomForestClassifier(n_estimators = 100)  
sklearn_forest.fit(train_data_cleaned.iloc[:,:-1], train_data_cleaned.iloc[:,-1])

RandomForestClassifier()

In [28]:
sklearn_forest_predicted_y = sklearn_forest.predict(test_data_cleaned)[:1459]

In [29]:
CalcAccuracy(list(test_labels["SalePrice"]), sklearn_forest_predicted_y)

45688.66962014401

In [31]:
number_of_trees_list = [x for x in range(1, 11)]
result_table = pd.DataFrame(index = number_of_trees_list, columns = ["Our Random Forest", "SKLearn Random Forest"])

for i in number_of_trees_list:
    forest = RF()
    # This takes a long time to run
    forest.build_model(train_data_cleaned, 0.1, i)
    forest_predicted_y = (forest.predict(test_data_cleaned))[:1459]
    our_forest_accuracy = CalcAccuracy(list(test_labels["SalePrice"]), forest_predicted_y)
    
    # Multiplying by 10 because otherwise the sklearn random forest classifier performs very bad
    sklearn_forest = RandomForestClassifier(n_estimators = (i * 10))  
    sklearn_forest.fit(train_data_cleaned.iloc[:,:-1], train_data_cleaned.iloc[:,-1])
    sklearn_forest_predicted_y = sklearn_forest.predict(test_data_cleaned)[:1459]
    sklearn_forest_accuracy = CalcAccuracy(list(test_labels["SalePrice"]), sklearn_forest_predicted_y)
    
    print("Done Training with " + str(i) + " number of trees in random forest!")
    # Add to dataframe here
    result_table.loc[i, ['Our Random Forest','SKLearn Random Forest']] = [our_forest_accuracy, sklearn_forest_accuracy]

Done Training with 1 number of trees in random forest!
Done Training with 2 number of trees in random forest!
Done Training with 3 number of trees in random forest!
Done Training with 4 number of trees in random forest!
Done Training with 5 number of trees in random forest!
Done Training with 6 number of trees in random forest!
Done Training with 7 number of trees in random forest!
Done Training with 8 number of trees in random forest!
Done Training with 9 number of trees in random forest!
Done Training with 10 number of trees in random forest!


In [32]:
print(result_table)

   Our Random Forest SKLearn Random Forest
1            46897.5               62694.3
2            48082.6               51305.8
3            44863.5               50356.3
4            43318.7               50138.4
5            48345.6               49119.8
6            41834.4               47469.5
7            41010.1               48471.7
8            39019.6               49020.3
9            41345.6               47740.9
10           40162.7               47034.1
